In [55]:
# == Imports ==
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from qiskit_aer import Aer
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import Statevector
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime.fake_provider import FakeOsaka

import numpy as np

In [56]:
# Generate dataset
X, y = make_moons(n_samples=500, noise=0.2, random_state=42)

# Standardise features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into test and training set
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

In [57]:
print(X_train[0:10])

[[ 0.86168207 -1.43528754]
 [-1.59128449 -0.18399934]
 [ 0.40686002  1.22397763]
 [ 1.6349996   0.17127311]
 [ 0.02739669  0.54293623]
 [ 1.44734317 -1.60513848]
 [-0.73674419  1.43222238]
 [ 0.24203334 -1.30093751]
 [ 2.04661675  0.47562793]
 [ 1.48511023  0.01210787]]


These is pretty simple data so we can angularly encode it.

In [58]:
def encode(x: np.ndarray) -> QuantumCircuit:
    qc = QuantumCircuit(len(x))
    for i in range(len(x)):
        qc.ry(x[i], i)
    return qc

In [59]:
# Variational Quantum Classifier
def vqc(reps: int, n_qubits: int) -> QuantumCircuit:
    # Build ansatz with alternating rotation blocks
    ansatz = TwoLocal(n_qubits, rotation_blocks=['ry', 'rx'], entanglement_blocks='cz',
                          entanglement='linear', reps=reps, insert_barriers=True)

    return ansatz

In [60]:
def predict(x, theta_vals):
    # Encode data
    qc = encode(x)
    
    # Assign params to ansatz and combine
    bound_ansatz = ansatz.assign_parameters(theta_vals)
    qc.compose(bound_ansatz, inplace=True)
    
    # Get statevector
    sv = Statevector.from_instruction(qc)
    
    # Compute probability of measuring |1> on qubit 0
    probs = sv.probabilities_dict()
    prob_1 = sum(v for k, v in probs.items() if k[-1] == '1')  # check last bit
    return prob_1

In [64]:
# Maybe update this, this could be better
def loss_fn(X, y, theta_vals):
    loss = 0
    for xi, yi in zip(X, y):
        prob = predict(xi, theta_vals)
        pred = prob # Predicted probability of class 1 (don't need deepcopy here because it's a float)
        loss += (pred - yi) ** 2
    return loss / len(X)

In [65]:
# Initialise problem circuit
n_qubits = 2
reps = 3
ansatz = vqc(reps, n_qubits)
param_vector = ParameterVector('θ', length=len(ansatz.parameters))

# Assign parameter vector to ansatz
param_dict = dict(zip(ansatz.parameters, param_vector))
ansatz.assign_parameters(param_dict, inplace=True)

# Simulator
sampler = Aer.get_backend('statevector_simulator')

In [67]:
# == TRAIN ==
theta = np.random.uniform(0, 2 * np.pi, len(param_vector))
lr = 0.1
for epoch in range(100):
    grad = np.zeros_like(theta)
    eps = 1e-4
    # Finite difference gradient
    for i in range(len(theta)):
        theta_plus = theta.copy()
        theta_plus[i] += eps
        theta_minus = theta.copy()
        theta_minus[i] -= eps

        grad[i] = (loss_fn(X_train, y_train, theta_plus) - loss_fn(X_train, y_train, theta_minus)) / (2 * eps)

    theta -= lr * grad
    train_loss = loss_fn(X_train, y_train, theta)
    print(f"Epoch {epoch:02d}: Loss = {train_loss:.4f}")

Epoch 00: Loss = 0.2507
Epoch 01: Loss = 0.2417
Epoch 02: Loss = 0.2330
Epoch 03: Loss = 0.2246
Epoch 04: Loss = 0.2167
Epoch 05: Loss = 0.2091
Epoch 06: Loss = 0.2019
Epoch 07: Loss = 0.1951
Epoch 08: Loss = 0.1888
Epoch 09: Loss = 0.1828
Epoch 10: Loss = 0.1773
Epoch 11: Loss = 0.1721
Epoch 12: Loss = 0.1672
Epoch 13: Loss = 0.1628
Epoch 14: Loss = 0.1586
Epoch 15: Loss = 0.1548
Epoch 16: Loss = 0.1512
Epoch 17: Loss = 0.1480
Epoch 18: Loss = 0.1449
Epoch 19: Loss = 0.1421
Epoch 20: Loss = 0.1396
Epoch 21: Loss = 0.1372
Epoch 22: Loss = 0.1350
Epoch 23: Loss = 0.1330
Epoch 24: Loss = 0.1311
Epoch 25: Loss = 0.1294
Epoch 26: Loss = 0.1278
Epoch 27: Loss = 0.1263
Epoch 28: Loss = 0.1250
Epoch 29: Loss = 0.1237
Epoch 30: Loss = 0.1225
Epoch 31: Loss = 0.1215
Epoch 32: Loss = 0.1205
Epoch 33: Loss = 0.1195
Epoch 34: Loss = 0.1187
Epoch 35: Loss = 0.1178
Epoch 36: Loss = 0.1171
Epoch 37: Loss = 0.1164
Epoch 38: Loss = 0.1157
Epoch 39: Loss = 0.1151
Epoch 40: Loss = 0.1145
Epoch 41: Loss =